In [1]:
## apply a transformer to classify protein families based on amino acid sequence

## read in kaggle dataset using their API
## this notebook is run within kaggle with filepaths pointing to data once it has been loaded
## into the kernel session. Change /kaggle/input/pfam-seed-random-split/ to your current directory
## requires Tensorflow version >2.2.0
## !kaggle datasets download -d googleai/pfam-seed-random-split

import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import math
import glob

def convertToNumber (s):
    return int.from_bytes(s.encode(), 'little')

def convertFromNumber (n):
    return n.to_bytes(math.ceil(n.bit_length() / 8), 'little').decode()


In [10]:
## read in training data
## tokenize protein sequences and family membership
def ReadInData (path):
    listprto=[]
    listfam=[]
    for file in glob.glob(path):
        df=pd.read_csv(file, sep=',',header=None)
        df = df.iloc[1: , :]
        listprto1 = list(df.iloc[:, 4])
        listprto = listprto + listprto1
        listfam = listfam + list(df.iloc[:,0])
    x_train = []
    for listval in listprto:
        x_train.append([ord(x) - 65 for x in listval])
    y_train = [convertToNumber(item) for item in listfam]
    return x_train , y_train

x_train , y_train = ReadInData("/kaggle/input/pfam-seed-random-split/random_split/train/data-*")
x_test , y_test = ReadInData("/kaggle/input/pfam-seed-random-split/random_split/dev/data-*")

In [11]:
## custom layer for transformer block
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [17]:
## prepare data for tensorflow and set parameters for number of tokens (number of proteins)
## and for number of positions of the protein to consider
vocab_size = 30
## maxlen = max([len(item) for item in x_train])
maxlen = 25

x_traink = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_valk = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

keys_list = list(set(y_train + y_test))
values_list = range(len(keys_list))
zip_iterator = zip(keys_list, values_list)
a_dictionary = dict(zip_iterator)

C = (pd.Series(y_train)).map(a_dictionary)
Y_train = list(C)

C1 = (pd.Series(y_test)).map(a_dictionary)
Y_test = list(C1)

In [18]:
## create embedding layer for amino acid and amino acid position 

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [19]:
## set hyperparameters and create model

embed_dim = 100  # embedding dimensions for each amino acid
num_heads = 8  # number of attention heads
ff_dim = 100  # hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(100, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(len(a_dictionary), activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
## compile and train model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_traink, np.asarray(Y_train).astype('float32'), batch_size=512, epochs=3, validation_data=(x_valk, np.asarray(Y_test).astype('float32'))
)


In [ ]:
## prepare test data
x_finaltest , y_finaltest = ReadInData("/kaggle/input/pfam-seed-random-split/random_split/test/data-*")
C1 = (pd.Series(y_finaltest)).map(a_dictionary)
Y_final = list(C1)

x_finalk = keras.preprocessing.sequence.pad_sequences(x_finaltest, maxlen=100)


In [ ]:
## predict protein family from test set
predicted_fin = model.predict(x_finalk)

In [ ]:
## calculate accuracy
sum(np.argmax(predicted_fin, axis=1) == Y_final)/len(Y_final)

In [ ]:
## make table with test prediction, true value, and whether they match 
final = [np.argmax(predicted_fin, axis=1), Y_final, np.argmax(predicted_fin, axis=1) == Y_final]

In [ ]:
## for each protein family calculate accuracy and total number of test sample
compare = [list(pd.DataFrame(final).T.groupby(1).mean().iloc[:,1]) , list(pd.DataFrame(final).T.groupby(1).size())]